In [2]:
import winsound as ws
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from bs4 import BeautifulSoup
import requests
import cv2
from sklearn.cluster import KMeans
from pathlib import Path

<h1>Get data from amazon</h1>
***
<p><b>Bookeeper</b> downloads the selected page's book from amazon and get some informations such cost and cover image</p>
<p><b>CoverColorPredict</b> discretize the color of every signle pixel of a selected cover and count the most frequent -> this is the color of the cover's book</p>

In [5]:
dataBooks = pd.read_csv("dataset\\books.csv")
#dataBooks = dataBooks[9700:10000]
dataBooks

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
5,6,11870085,11870085,16827462,226,525478817,9.780525e+12,John Green,2012.0,The Fault in Our Stars,...,2346404,2478609,140739,47994,92723,327550,698471,1311871,https://images.gr-assets.com/books/1360206420m...,https://images.gr-assets.com/books/1360206420s...
6,7,5907,5907,1540236,969,618260307,9.780618e+12,J.R.R. Tolkien,1937.0,The Hobbit or There and Back Again,...,2071616,2196809,37653,46023,76784,288649,665635,1119718,https://images.gr-assets.com/books/1372847500m...,https://images.gr-assets.com/books/1372847500s...
7,8,5107,5107,3036731,360,316769177,9.780317e+12,J.D. Salinger,1951.0,The Catcher in the Rye,...,2044241,2120637,44920,109383,185520,455042,661516,709176,https://images.gr-assets.com/books/1398034300m...,https://images.gr-assets.com/books/1398034300s...
8,9,960,960,3338963,311,1416524797,9.781417e+12,Dan Brown,2000.0,Angels & Demons,...,2001311,2078754,25112,77841,145740,458429,716569,680175,https://images.gr-assets.com/books/1303390735m...,https://images.gr-assets.com/books/1303390735s...
9,10,1885,1885,3060926,3455,679783261,9.780680e+12,Jane Austen,1813.0,Pride and Prejudice,...,2035490,2191465,49152,54700,86485,284852,609755,1155673,https://images.gr-assets.com/books/1320399351m...,https://images.gr-assets.com/books/1320399351s...


In [15]:
class Bookeeper:
    
    def __init__(self, isbn10, bookid):
        self.bookid = bookid
        self.isbn10 = str(isbn10).zfill(10)
        self.urlbook = "https://www.amazon.com/gp/search/ref=sr_adv_b/?search-alias=stripbooks&unfiltered=1&field-keywords=&field-author=&field-title=&field-isbn="+self.isbn10+"&field-publisher=&node=&field-p_n_condition-type=&p_n_feature_browse-bin=&field-age_range=&field-language=&field-dateop=During&field-datemod=&field-dateyear=&sort=relevanceexprank"
        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
        self.page = requests.get(self.urlbook, headers=headers)
        self.bs = BeautifulSoup( self.page.text, "html.parser")
            
    #get the list price
    def getListPrice(self):
        
        try:
            costo1 = float(self.bs.findAll("span", attrs = {"class": "a-size-base-plus a-color-secondary a-text-strike"})[0].text.replace("$", ""))
        except:
            costo1 = -1
        try:
            costo2 = float(self.bs.findAll("span", attrs = {"class": "a-offscreen"})[0].text.replace("$", ""))
        except:
            costo2 = -1
        try:
            costo3 = float(self.bs.findAll("span", attrs = {"class": "a-size-base a-color-base"})[0].text.replace("$", ""))
        except:
            costo3 = -1
            
        self.cost = max([costo1, costo2, costo3])
        
        if( self.cost < 0):
            raise ValueError('Cannot keep cost of the book')
            
        return self.cost
    
    
    #gets the COVER images
    def saveImageCover(self, location = "sampleImages\\", url = None):
        if(url == None):
            #gets source
            self.sourceCover = self.bs.findAll("img", attrs={"class" : "s-access-image cfMarker"})[0]["src"]
        else:
            self.sourceCover = url
        #gets image data
        self.img_data = requests.get(self.sourceCover).content
        with open(location+str(self.bookid)+".jpg", 'wb') as handler:
            handler.write(self.img_data)

In [16]:
class CoverColorPredict:
    #setting up palette
    palette = np.array([[0,0,0], [255,255,255], [255,0,0], [0, 255, 0], [0,0,255], [255,255,0], [255, 0, 255], [0, 255, 255], [153, 51, 0], [255,102,0]], np.float64)
    palettelabels = ["Black", "White", "Red", "Green", "Blue", "Yellow", "Violet", "Cyan", "Brown", "Orange", ];
    
    
    #use id_book like img_name
    def __init__(self, img_name):
        # load the image and convert it from BGR to RGB so that
        # we can dispaly it with matplotlib
        self.img_name = img_name
        self.image = cv2.imread('D:\\foto\\sampleImages\\'+img_name+'.jpg')
        self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB) 
        # reshape the image to be a list of pixels
        self.image = self.image.reshape((self.image.shape[0] * self.image.shape[1], 3))
        # cluster the pixel intensities
        self.clt = KMeans(n_clusters = len(CoverColorPredict.palette), init=CoverColorPredict.palette, n_init=1)
        self.clt.fit(self.image)
        
        
    #returns index of the palette and the name of color predicte
    def predictColor(self):
        #summurize data of the image
        self.labels = []
        self.results = []
        self.r = []
        self.g = []
        self.b = []
        for a in range(len(CoverColorPredict.palette)):
            self.labels.append(a)
            self.results.append(0)
            self.r.append(int(self.clt.cluster_centers_[a][0]))
            self.g.append(int(self.clt.cluster_centers_[a][1]))
            self.b.append(int(self.clt.cluster_centers_[a][2]))

        for i in self.clt.labels_:
            self.results[i] = self.results[i]+1
        #create dataset to summarize data
        self.colorresults = pd.DataFrame(data = { "labels" : self.labels, "n_pixels": self.results, "value_r" : self.r, "value_g": self.g, "value_b": self.b})
        self.maxcolor = self.colorresults["n_pixels"].idxmax()
        self.maincolor = self.clt.cluster_centers_[self.maxcolor].astype(int)
        self.predictedcolor = self.clt.predict([self.maincolor])
        #returns palette id and the label
        return self.predictedcolor, CoverColorPredict.palettelabels[self.predictedcolor[0]]
        
    
    

<p>This section takes every book's ISBN and id and save informations in a different dataset on disk</p>

In [5]:
pricelist = []
colorlist = []
coloridlist = []

for index, row in dataBooks.iterrows():
    print("Elbarating id: "+str(row["id"])+" bookid: "+str(row["book_id"])+" isbn10 "+str(row["isbn"]))
    maxattemps = 1
    for i in range(maxattemps):
        if(i != 0):
            print("Retry id: "+str(row["book_id"])+" isbn10 "+str(row["isbn"])) 
        try:

            bk = Bookeeper(row["isbn"], row["book_id"])
            bk.saveImageCover(url = str(row["image_url"]))
            price = bk.getListPrice()
            pricelist.append(price)
            break;
        except:
            print("Error in bookKeeper id: "+str(row["book_id"])+" isbn10 "+str(row["isbn"]))
            if(i == maxattemps-1):
                pricelist.append(None)
    try:
        
        col = CoverColorPredict(str(row["book_id"]))
        colorpredicted = col.predictColor()
        colorlist.append(colorpredicted[1])
        coloridlist.append(colorpredicted[0][0])
    except:
        print("Error in color predict id: "+str(row["book_id"])+" isbn10 "+str(row["isbn"]))
        colorlist.append(None)
        coloridlist.append(None)
    
print("DONE")
ws.Beep(500,500)
dataBooks["price(USD)"] = pricelist
dataBooks["color_name"] = colorlist
dataBooks["color_id"] = coloridlist


Elbarating id: 9701 bookid: 25067046 isbn10 1484724984
Elbarating id: 9702 bookid: 128048 isbn10 618250743
Elbarating id: 9703 bookid: 22474626 isbn10 nan
Error in bookKeeper id: 22474626 isbn10 nan
Elbarating id: 9704 bookid: 95660 isbn10 316115002
Elbarating id: 9705 bookid: 17794657 isbn10 nan
Error in bookKeeper id: 17794657 isbn10 nan
Elbarating id: 9706 bookid: 18243828 isbn10 nan
Error in bookKeeper id: 18243828 isbn10 nan
Elbarating id: 9707 bookid: 171201 isbn10 571198775
Elbarating id: 9708 bookid: 261161 isbn10 316115045
Elbarating id: 9709 bookid: 13031445 isbn10 1451668201
Elbarating id: 9710 bookid: 397867 isbn10 333761189
Elbarating id: 9711 bookid: 104088 isbn10 451458265
Elbarating id: 9712 bookid: 17333174 isbn10 425270696
Elbarating id: 9713 bookid: 15761241 isbn10 nan
Error in bookKeeper id: 15761241 isbn10 nan
Elbarating id: 9714 bookid: 23196 isbn10 553344862
Elbarating id: 9715 bookid: 701738 isbn10 1400096685
Elbarating id: 9716 bookid: 586516 isbn10 425182134
E

Elbarating id: 9842 bookid: 24019187 isbn10 1250058902
Elbarating id: 9843 bookid: 71867 isbn10 439858054
Elbarating id: 9844 bookid: 17235347 isbn10 nan
Error in bookKeeper id: 17235347 isbn10 nan
Elbarating id: 9845 bookid: 13360101 isbn10 345533232
Elbarating id: 9846 bookid: 1099097 isbn10 6640613
Elbarating id: 9847 bookid: 20685373 isbn10 307720659
Elbarating id: 9848 bookid: 61899 isbn10 671578855
Elbarating id: 9849 bookid: 36638 isbn10 385327900
Elbarating id: 9850 bookid: 17563539 isbn10 nan
Error in bookKeeper id: 17563539 isbn10 nan
Elbarating id: 9851 bookid: 784222 isbn10 1421513242
Elbarating id: 9852 bookid: 31266 isbn10 752858653
Elbarating id: 9853 bookid: 690955 isbn10 1596430699
Elbarating id: 9854 bookid: 15398 isbn10 743225740
Elbarating id: 9855 bookid: 5266667 isbn10 60874198
Elbarating id: 9856 bookid: 268955 isbn10 679824243
Elbarating id: 9857 bookid: 10461 isbn10 1573225053
Elbarating id: 9858 bookid: 383196 isbn10 9649007903
Elbarating id: 9859 bookid: 2677

Elbarating id: 9982 bookid: 12444298 isbn10 385344422
Elbarating id: 9983 bookid: 8539798 isbn10 2849659266
Elbarating id: 9984 bookid: 764165 isbn10 140255087
Elbarating id: 9985 bookid: 19688 isbn10 425176932
Elbarating id: 9986 bookid: 183092 isbn10 310257689
Elbarating id: 9987 bookid: 8087038 isbn10 312651198
Elbarating id: 9988 bookid: 129237 isbn10 674017722
Elbarating id: 9989 bookid: 13489518 isbn10 nan
Error in bookKeeper id: 13489518 isbn10 nan
Elbarating id: 9990 bookid: 294081 isbn10 140143912
Elbarating id: 9991 bookid: 101094 isbn10 385425139
Elbarating id: 9992 bookid: 13616278 isbn10 575113294
Elbarating id: 9993 bookid: 4936457 isbn10 224079948
Elbarating id: 9994 bookid: 4769651 isbn10 810983559
Elbarating id: 9995 bookid: 15613 isbn10 1416523723
Elbarating id: 9996 bookid: 7130616 isbn10 441019455
Elbarating id: 9997 bookid: 208324 isbn10 067973371X
Elbarating id: 9998 bookid: 77431 isbn10 039330762X
Elbarating id: 9999 bookid: 8565083 isbn10 61711527
Elbarating id:

In [6]:
dataBooks.to_csv("dataset\\new\\books25.csv")

***
<p>Following section check data, and check white books ( falsi positivi ) </p>

In [17]:
#Check all files
listalldata = []
for i in range(1,26):
    data = pd.read_csv("dataset\\new\\books"+str(i)+".csv")
    listalldata.append(data)
alldata = pd.concat(listalldata, ignore_index=True)
alldata = alldata.drop(axis = 1, columns=["Unnamed: 0", "color_id"])
#drop double books
axes = alldata["book_id"].value_counts().axes[0].tolist()
values = alldata["book_id"].value_counts().values.tolist()

for i in range(len(values)):
    if(values[i] > 1):
        indexes = alldata[alldata.book_id == axes[i]].index.tolist()
        alldata = alldata.drop(indexes[1:])
alldata[alldata.id ==9881]

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,price(USD),color_name
11678,9881,511684,511684,499669,40,440863686,9.780441e+12,"Jacqueline Wilson, Nick Sharratt",1999.0,The Illustrated Mum,...,321,290,977,3319,3554,3201,https://images.gr-assets.com/books/1320392229m...,https://images.gr-assets.com/books/1320392229s...,15.41,Cyan


In [18]:
#not found books images 
whitebookid = alldata[alldata.color_name == "White"].dropna(axis=0, subset=["isbn"])["book_id"].values.tolist()
listwhite = []
for whiteid in whitebookid:
    if(not Path("sampleImages\\"+str(whiteid)+".jpg").is_file()):
        listwhite.append(whiteid)

print("White to download: "+str(len(listwhite)))
        
for bookid in listwhite[:500]:
    try:
        isbn = alldata[alldata.book_id == bookid]["isbn"].values[0]
        print("Elaborating "+str(bookid)+" whith isbn "+str(isbn))
        bk = Bookeeper(isbn, bookid)
        bk.saveImageCover()
        img = cv2.imread("sampleImages\\"+str(bookid)+".jpg")
        crop_img = img[0:218, 45:173]
        cv2.imwrite("sampleImages\\"+str(bookid)+".jpg", crop_img)
    except:
        print("Error in bookKeeper id: "+str(bookid))
        pricelist.append(None)
print("DONE")

White to download: 107
Elaborating 9010 whith isbn 806527285
Error in bookKeeper id: 9010


NameError: name 'pricelist' is not defined

In [ ]:
nowhite = alldata[alldata.color_name != "White"]
nowhite
white = alldata[alldata.color_name == "White"]
white

In [ ]:
white = white.drop(axis = 1, columns=["color_name"])
colorlist = []
for index, row in white.iterrows():
    print("Elbarating id: "+str(row["id"])+" bookid: "+str(row["book_id"])+" isbn10 "+str(row["isbn"]))
    try:
        col = CoverColorPredict(str(row["book_id"]))
        colorpredicted = col.predictColor()
        colorlist.append(colorpredicted[1])
    except:
        print("Error in color predict id: "+str(row["book_id"])+" isbn10 "+str(row["isbn"]))
        colorlist.append(None)
        coloridlist.append(None)

newwhite = white.assign(color_name = colorlist)
newalldata = pd.concat([nowhite, newwhite], ignore_index =True)

In [19]:
alldata.to_csv("dataset\\finals\\allbooks.csv")